In [1]:
import tkinter
import pyautogui as pag
import cv2
import numpy as np
import mediapipe as mp
import time
import os
import sys  # zahvat
import torch
import torch.nn as nn
import numpy as np
import json
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import os    
import torch.nn.functional as F
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [2]:
class Dataset(Dataset):
    
    def __init__(self, *args): #получаем n массивов данных
        cnt, labels, temp =0, [], []
        self.data = []
        for data in args: #выделяем по одному из них
            self.data.extend(torch.tensor(data).type(torch.float32)) # преобразуем каждый в тензор и добавляем к общим данным
#             print(len(data))
            for j in range(len(data)):
                labels.append(cnt) #присовим номера каждому подмассиву для проверки
            cnt += 1
        self.labels = labels
            
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
    
    def __len__(self):
        return len(self.data)

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(42, 200)
        self.fc2 = nn.Linear(200, 200)
        self.fc3 = nn.Linear(200, 32)
        self.fc4 = nn.Linear(32, 2)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [4]:
gesture = {0: "Fist",
           1:"Open palm",
           2:"Окей",
           3:"Жест 4",
          4:"Жест 5",
          5:"Жест 6"}

def Result_chek(neuro_res):
    znak_num=len(neuro_res)
    global gesture
    answ = {}
    for el in range(znak_num):
        answ[el] = neuro_res[el]
    res = answ.keys()
#     print(answ.keys())
#     print(list(answ.values()).index(max(neuro_res)))
#     print(list(answ.keys())[list(answ.values()).index(max(neuro_res))])
    return gesture[list(answ.values()).index(max(neuro_res))]#("Жест на экране:  "

In [5]:
f = open('dataset/stop2.txt').readline()
f = json.loads(f)
# print(len(f))

f = open('dataset/kulaksgat2.txt').readline()
f = json.loads(f)
# print(len(f))

In [6]:
data, temp = [],[]
gests = os.listdir('dataset')
for gest in gests:
    data = []
#     print(gest)
    f = open('dataset/' + str(gest)).readline()
    f = json.loads(f)
    for i in range (len(f)):
        data.append(f[i])
    temp.append(data)
data = np.array(temp, dtype=object)

In [7]:
dataset = Dataset(*data)
dataloader = DataLoader(dataset, shuffle=True, batch_size = 16)

In [55]:
net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

for epoch in range(10):
    running_loss = 0.0
    for samples, labels in dataloader:
        #print(samples,labels)
        pred = net(samples)
        loss = criterion(pred, labels)
        
        
        loss.backward()
        optimizer.step()
        
        optimizer.zero_grad()
        running_loss += loss.item()
#     print(running_loss)
print('Finished Training', running_loss)

Finished Training 8.359541530555958e-06


In [65]:
import tkinter
import pyautogui as pag
import cv2
import keyboard

def smooth_mouse(x,y, ox, oy, sens):
    speed_x = x - ox if x - ox > 0 else ox - x
    speed_y = y - oy if y - oy > 0 else oy- y
    dur = 1 - (max(speed_y, speed_x))
#     print(sens)
    shift = (x-ox) * sens[0], (y - oy) * sens[1] * 0,5
#     print(shift)
    for i in range(5):
        print(pag.position())
        pag.moveTo(shift[0]//5 + pag.position()[0], shift[1]//5 + pag.position()[1])
#     pag.moveTo(shift[0], shift[1])


def mouse_control(b):

#     cap = cv2.VideoCapture(0) 
#     cap.set(3, 640) # Width
#     cap.set(4, 480) # Lenght
#     cap.set(10, 100) # Brightness
#     mpHands = mp.solutions.hands
#     hands = mpHands.Hands(False)

    sensetive = 1
    pag.moveTo(1000,500)
    pag.FAILSAFE = False
    weigth, height = pag.size()
    print("-----",weigth,"----")
    koef = [weigth*sensetive,height*sensetive]
    pred_x, pred_y, now_x, now_y = 0,0,0,0    
    success, img = cap.read()
    img = cv2.flip(img,1) # Mirror flip
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    if results.multi_hand_landmarks:
#         print(results.multi_hand_landmarks[8])
        for handLms in results.multi_hand_landmarks:
            try:
                lm = handLms.landmark[8]
                now_x, now_y = lm.x, lm.y
            except IndexErrror:
                pass  


    while Result_chek(net(torch.tensor(b))) != "Fist":
        success, img = cap.read()
        img = cv2.flip(img,1) # Mirror flip
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(imgRGB)
        if results.multi_hand_landmarks:
            for handLms in results.multi_hand_landmarks:
                lm = handLms.landmark[8]
                now_x, now_y = lm.x, lm.y
                x,y = pred_x - now_x,  now_y - pred_y 
                
                b = []
                for id, lmt in enumerate(handLms.landmark):
                    b.append(lmt.x)
                    b.append(lmt.y)
#                 cv2.putText(img, Result_chek(net(torch.tensor(b))), (200,30), cv2.FONT_HERSHEY_PLAIN,2, (25,20,0),2)
                    
                #func place
                try:
                    smooth_mouse(now_x, now_y, pred_x, pred_y, koef)
                except BaseException:
                    print("out of border")
    #                 cap.release()
    #                 cv2.waitKey(1)   
    #                 break
                    pag.moveTo(1000,500)
                pred_x, pred_y = now_x, now_y
        if keyboard.is_pressed('esc'):  # if key 'esc' is pressed
            cap.release()
            cv2.waitKey(1)   
            break

In [67]:
import cv2
import numpy as np
import mediapipe as mp
import time
import os
import sys 


# Подключаем камеру
cap = cv2.VideoCapture(0) 
cap.set(3, 640) # Width
cap.set(4, 480) # Lenght
cap.set(10, 100) # Brightness

mpHands = mp.solutions.hands
hands = mpHands.Hands(False)
npDraw = mp.solutions.drawing_utils
pTime = 0
cTime = 0
data = []
 #Зацикливаем получение кадров от камеры
while True: 
    success, img = cap.read()
    img = cv2.flip(img,1) # Mirror flip
    cx8=0
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    pred2 = []
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
#             print("handLms  ", handLms.landmark[1])
            b = []
            for id, lm in enumerate(handLms.landmark):
                h,w,c = img.shape
                cx, cy = int(lm.x*w), int(lm.y*h)
                b.append(lm.x)
                b.append(lm.y)
                if id == 8:
                    cx8 = cx
                # print(id, lm)
                if  id == 8: #id == 12
                    cv2.circle(img, (cx,cy),10,(255,0,255),cv2.FILLED)
            if len(b) == 42:
                pred = net(torch.tensor(b))
                pred2 = pred.detach().numpy()
            cv2.putText(img, Result_chek(net(torch.tensor(b))), (200,30), cv2.FONT_HERSHEY_PLAIN,2, (25,20,0),2)
            
            if Result_chek(net(torch.tensor(b))) != "Fist": # управление мышью
                mouse_control(b)
                
                
            npDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)
            
    

    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    exist = [float(i) for i in pred2]
#     cv2.putText(img, str(exist),(10,30), cv2.FONT_HERSHEY_PLAIN, 2, (255,0,0), 2) # ФреймРейт
#     cv2.putText(img, str(cx8),(100,30), cv2.FONT_HERSHEY_PLAIN, 2, (255,0,0), 2)
    cv2.imshow('python', img)
    if cv2.waitKey(20) == 27: # exit on ESC
        break
        
cv2.destroyWindow("python")
cap.release()
cv2.waitKey(1)
# print(a) # a is massive of data we need


-1

In [44]:
for i in range(10):
    pag.hotkey('ctrl', 'shift', 'esc')

In [37]:
print(1)

1
